# 数据预处理

In [1]:
import pandas as pd
import os
import glob

## 读取数据

In [20]:
# use glob to get all the csv files 
# in the folder
path = r"./data"
csv_files = glob.glob(os.path.join(path, "*.csv"))
df_list = []
for f in csv_files:
    df = pd.read_csv(f)      
    df_list.append(df.iloc[:,-2:])

df_list[5]

,看完这集我认识了一个单词decoy,0
0,他桌子上没按钮吗（doge,2
1,真正的瑞克从来都不会掺和进自己的烂摊子,2
2,难道不是她认不出那个才是自己弟弟吗？,3
3,好家伙 梦幻联动,0
4,瑞克嘴角没有口水了,1
...,...,...
1761,第一播放 五十个赞,0
1762,刚刚,0
1763,第一?,0
1764,唉,1


In [9]:
pd.read_csv("sample.csv")

,"161.17400,1,25,16777215,1622884323,0,418a81da,50196459321032711",555难哭了 老师我爱你
0,"3856.31700,1,25,16777215,1622861110,0,5f97eab3...",怎么解的方程
1,"3062.90400,1,25,16777215,1622812788,0,1577f985...",取对数，把Z解出来在对x求导也行吧
2,"2800.27400,1,25,16777215,1622812565,0,1577f985...",哈哈
3,"1127.09200,1,25,16777215,1622791383,0,dee97275...",那我走？
4,"1096.46000,1,25,16777215,1622791352,0,dee97275...",妙啊
...,...,...
3277,"1294.70300,1,25,16777215,1615189078,0,48954e6a...",全部材料
3278,"1176.82400,1,25,16777215,1615185368,0,2f623d31...",6+全部材料
3279,"1275.74000,1,25,16777215,1615173504,0,8b187c67...",全部
3280,"1174.53900,1,25,16777215,1615158629,0,c02f439e...",全部材料
